In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

%matplotlib inline

In [ ]:
hub.KerasLayer(
    "https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/035-128-classification/2",
    trainable=True,
    arguments=dict(batch_norm_momentum=0.997),
)

In [ ]:
IMAGE_SHAPE = (224, 224)
classifier = tf.keras.Sequential(
    [
        hub.KerasLayer(
            "https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/035-128-classification/2",
            input_shape=IMAGE_SHAPE + (3,),
        )
    ]
)